## Step 1: Define the LiH Molecule

LiH equilibrium bond length is approximately 1.595 Å


In [2]:
# Define LiH molecule
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
driver = PySCFDriver(
    atom="Li 0 0 0; H 0 0 1.595",  # LiH with equilibrium bond length
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

problem = driver.run()
print(f"Number of spatial orbitals: {problem.num_spatial_orbitals}")
print(f"Number of particles: {problem.num_particles}")
print(f"Nuclear repulsion energy: {problem.nuclear_repulsion_energy:.6f} Ha")


Number of spatial orbitals: 6
Number of particles: (2, 2)
Nuclear repulsion energy: 0.995318 Ha


In [3]:
# Consolidated imports for the notebook
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_algorithms import VQE, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import SLSQP, SPSA
from qiskit.primitives import StatevectorEstimator
import matplotlib.pyplot as plt
import numpy as np

print("Imported Qiskit Nature, Qiskit Algorithms, and plotting libraries.")


Imported Qiskit Nature, Qiskit Algorithms, and plotting libraries.


In [4]:
# Get Hamiltonian
hamiltonian = problem.hamiltonian

# Get fermionic operator
second_q_op = hamiltonian.second_q_op()
print(f"Fermionic operator has {len(second_q_op)} terms")
print(f"Number of spin orbitals: {second_q_op.num_spin_orbitals}")


Fermionic operator has 1860 terms
Number of spin orbitals: 12


## Step 3: Map to Qubit Operator (Jordan-Wigner)


In [5]:
# Create mapper and map to qubits
mapper = JordanWignerMapper()
qubit_op = mapper.map(second_q_op)

print(f"Qubit operator has {len(qubit_op)} terms")
print(f"Number of qubits required: {qubit_op.num_qubits}")


Qubit operator has 631 terms
Number of qubits required: 12


## Step 4: Exact Solution (NumPy Solver)


In [6]:
# Exact solver
numpy_solver = NumPyMinimumEigensolver()
numpy_result = numpy_solver.compute_minimum_eigenvalue(qubit_op)

exact_electronic = numpy_result.eigenvalue.real
exact_total = exact_electronic + problem.nuclear_repulsion_energy

print("Exact Solution (NumPy):")
print(f"  Electronic energy: {exact_electronic:.8f} Ha")
print(f"  Total energy:      {exact_total:.8f} Ha")


Exact Solution (NumPy):
  Electronic energy: -8.87771957 Ha
  Total energy:      -7.88240193 Ha


## Step 5: VQE with UCCSD Ansatz


In [7]:
# Create UCCSD ansatz with Hartree-Fock initial state
ansatz = UCCSD(
    problem.num_spatial_orbitals,
    problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
    ),
)

print(f"UCCSD ansatz has {ansatz.num_parameters} parameters")
print(f"Circuit depth: {ansatz.decompose().depth()}")


UCCSD ansatz has 92 parameters
Circuit depth: 93


In [8]:
# Prepared VQE callback and helpers — do NOT run the blocking VQE here.
# Instead use the 'Run VQE in a background thread' cell below to start optimization.
from qiskit_algorithms.optimizers import SPSA
import time

# Shared structures used by the background runner and monitor
convergence_history = []
start_time = None  # will be set by the background runner

def callback(eval_count, parameters, mean, std):
    """Callback to track progress"""
    convergence_history.append(mean)
    if start_time is not None:
        elapsed = time.time() - start_time
    else:
        elapsed = 0
    if len(convergence_history) % 10 == 0:
        print(f"Eval {len(convergence_history)}: Energy = {mean:.6f} Ha (Time: {elapsed:.1f}s)")

print("Prepared VQE callback and convergence tracking.")
print("To run VQE, run the 'Run VQE in a background thread' cell (it starts the optimizer without blocking the kernel).")


Prepared VQE callback and convergence tracking.
To run VQE, run the 'Run VQE in a background thread' cell (it starts the optimizer without blocking the kernel).


In [9]:
# ETA helper: estimate remaining time for VQE (run while VQE is running or after some iterations)
import time
current_evals = len(convergence_history) if 'convergence_history' in globals() else 0
elapsed = time.time() - start_time if 'start_time' in globals() else None
# Notebook uses SPSA(maxiter=300) by default; SPSA typically does ~2 energy evals per iteration
maxiter = 300
expected_evals_spsa = maxiter * 2  # adjust if you changed maxiter or optimizer
if elapsed is None or current_evals == 0:
    print('Not enough data to estimate ETA yet. Wait for a few evaluations and run this cell again.')
else:
    avg_sec_per_eval = elapsed / current_evals
    remaining = max(0, expected_evals_spsa - current_evals)
    eta_sec = remaining * avg_sec_per_eval
    print(f'Elapsed: {elapsed:.1f}s for {current_evals} evals — avg {avg_sec_per_eval:.2f}s/eval')
    print(f'Estimated remaining evals (SPSA assumption): {remaining} — ETA: {eta_sec/60:.1f} min ({eta_sec:.0f}s)')
    print('If you expect fewer total evals, set expected_evals accordingly (e.g., to maxiter instead of 2*maxiter).')


TypeError: unsupported operand type(s) for -: 'float' and 'NoneType'

In [ ]:
# Run VQE in a background thread so other cells (like monitors) can run
import threading

def run_vqe_background(maxiter=300):
    global vqe_result, convergence_history, start_time, vqe_solver, vqe_thread_maxiter
    from qiskit_algorithms.optimizers import SPSA
    convergence_history = []
    start_time = time.time()
    vqe_thread_maxiter = maxiter
    vqe_solver = VQE(StatevectorEstimator(), ansatz, SPSA(maxiter=maxiter), callback=callback)
    vqe_solver.initial_point = [0.0] * ansatz.num_parameters
    vqe_result = vqe_solver.compute_minimum_eigenvalue(qubit_op)
    print("VQE background run completed")

# Start background thread (daemon=True so it won't block kernel exit)
thread = threading.Thread(target=run_vqe_background, kwargs={'maxiter':300}, daemon=True)
thread.start()
print("VQE started in background thread. Use the monitor cell to check progress.")


In [ ]:
# Live monitor: polls convergence_history and reports ETA while VQE runs in background
import time

def live_monitor(poll_interval=10):
    while True:
        current_evals = len(convergence_history) if 'convergence_history' in globals() else 0
        elapsed = time.time() - start_time if 'start_time' in globals() else None
        if elapsed is None or current_evals == 0:
            print('Waiting for first evaluations...')
        else:
            avg = elapsed / current_evals
            expected_evals = (vqe_thread_maxiter * 2) if 'vqe_thread_maxiter' in globals() else 300*2
            remaining = max(0, expected_evals - current_evals)
            eta = remaining * avg
            print(f'Elapsed {elapsed:.1f}s, evals {current_evals}, avg {avg:.2f}s/eval — ETA: {eta/60:.1f} min ({eta:.0f}s), remaining evals {remaining}')

        # Exit when background thread is finished (if thread variable exists)
        if 'thread' not in globals() or not thread.is_alive():
            print('VQE thread not running or finished; exiting monitor.')
            break
        time.sleep(poll_interval)

# Run the monitor (adjust poll_interval if desired)
live_monitor(poll_interval=10)


## QEB (Qubit Excitation-Based) Ansatz Implementation

### Physics Background:
**QEB vs UCCSD:**
- **UCCSD**: Uses fermionic excitation operators with Jordan-Wigner strings → many gates
- **QEB**: Uses qubit-space Givens rotations → preserves particle/spin symmetry without JW strings
- Key advantage: ~20x fewer CNOT gates for high-rank excitations, better trainability

**Key Concept:** Givens rotations are particle-conserving unitaries that rotate between states with equal particle count:
- SingleExcitation(θ, [i,j]): Swap electron between qubits i and j
- DoubleExcitation(θ, [i,j,k,l]): Swap two electrons between orbital pairs (i,j) and (k,l)

**Symmetry Preservation:** Although qubit excitations don't obey fermionic anticommutation relations, they preserve particle number and S_z symmetry (same as fermionic excitations).

### Implementation Approach:
1. Build excitation pool (singles + optional doubles)
2. Create layers of Givens rotations
3. Optional repetition depth L for increased expressibility
4. Initialize from Hartree-Fock state for faster convergence